### 1. Import Modules

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, losses, callbacks, applications, models
import numpy as np
import matplotlib.pyplot as plt
import os, datetime

### 2. Load Model

In [ ]:
model = keras.models.load_model(f"{os.getcwd()}/runs/model1.h5")


### 3. Splitting Dataset Similar To Training File To Use Test Dataset

In [ ]:
PATH = f"{os.getcwd()}/Datasets"

BATCH_SIZE = 27
IMG_SIZE = (160, 160)

dataset = tf.keras.utils.image_dataset_from_directory(PATH,
                                                        shuffle=True,
                                                        seed=42,
                                                        batch_size=BATCH_SIZE,
                                                        image_size=IMG_SIZE)

class_names = dataset.class_names

batches_train_test = tf.data.experimental.cardinality(dataset)
train_dataset = dataset.skip(batches_train_test//5)
test_dataset_before_split = dataset.take(batches_train_test//5)

batches_test_val = tf.data.experimental.cardinality(test_dataset_before_split)
test_dataset = test_dataset_before_split.take(batches_test_val//2)
validation_dataset = test_dataset_before_split.skip(batches_test_val//2)

print(len(dataset))
print(len(train_dataset))
print(len(test_dataset))
print(len(validation_dataset))

### 4. Model Deployment

In [ ]:
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch)
prediction_indexes = np.argmax(predictions, axis=1)
label_map = {i:names for i, names in enumerate(class_names)}
prediction_list = [label_map[i] for i in prediction_indexes]
label_list = [label_map[i] for i in label_batch]

plt.figure(figsize=(10,10))
for i in range(9):
    ax = plt.subplot(3, 3, i+1)
    plt.imshow(image_batch[i].astype("uint8"))
    if prediction_list[i] == label_list[i]:
        mark = "✔️"
    else:
        mark = "❌"
    plt.title(f"Prediction: {prediction_list[i]}\nLabel: {label_list[i]}    {mark}")
    plt.axis("off")
    plt.grid("off")